Gerar grafos

In [1]:
import networkx as nx
import random
import numpy as np

def generate_random_graph(num_graph, num_node_min, num_node_max, prob_min, prob_max):
    graphs = []
    for i in range(num_graph):
        num_nodes = random.randint(num_node_min, num_node_max)
        edge_prob = random.uniform(prob_min, prob_max)
        graph = nx.gnp_random_graph(num_nodes, edge_prob)
        graphs.append(graph)
    return graphs

Funções para escrever no arquivo

In [2]:
def escrever_estado(matriz_adj, clique_atual, movimento):
    estado = []
    for linha in matriz_adj:
        estado.append("AA " + ', '.join(map(str, linha)) + "\n")
    estado.append("AA cliqueatual " + ' '.join(map(str, clique_atual)) + "\n")
    estado.append(f"AA movimento {movimento}\n")
    return estado

In [3]:
def copiar(arquivo, estados):
  with open(f"instancias/0{arquivo}", 'w') as f:
      for estado in estados:
          f.writelines(estado)

Coloração

In [4]:
def coloracao_gulosa(G, order):
    color = {}
    classes = []
    for v in order:
        # Cores disponíveis
        available = [True] * G.number_of_nodes()
        for u in G[v]:
            if u in color:
                available[color[u]] = False
        for i in range(G.number_of_nodes()):
            if available[i]:
                color[v] = i
                if i < len(classes):
                    classes[i].append(v)
                else:
                    classes.append([v])
                break
    return classes, color

In [5]:
def two_phase_coloring(G, order):
    # Fase 1: Coloração inicial (greedy)
    order2 = sorted(order, key=lambda v: G.degree(v), reverse=True)
    classes, initial_coloring = coloracao_gulosa(G, order2)
    
    # Fase 2: Coloração na ordem inversa da inicial
    reverse_order = sorted(initial_coloring.keys(), key=lambda x: initial_coloring[x], reverse=True)
    second_classes, second_coloring = coloracao_gulosa(G, reverse_order)
    
    return second_classes

Encontrar clique máxima

In [6]:
def atualizar_matriz_adjacencia(matriz_adj, v):

    nova_matriz_adj = matriz_adj.copy()
    for i in range(len(nova_matriz_adj)):
        if nova_matriz_adj[v][i] == 0 and i != v:
            nova_matriz_adj[i, :] = 0
    # Atribuir 0 a todas as arestas do vértice v
    nova_matriz_adj[v, :] = 0
    return nova_matriz_adj

In [7]:
def bb_clique(G, V, clique_atual, best_clique, arquivo, estados_escritos, adj_matriz):
  if len(V) == 0:
    if len(clique_atual) > len(best_clique):
      best_clique.clear()
      best_clique.extend(clique_atual)
      copiar(arquivo, estados_escritos)
  else:
    classes = two_phase_coloring(G, list(V))
    limite_superior = len(classes)

    if len(clique_atual) + limite_superior > len(best_clique):
        V = sorted(V, key=lambda v: G.degree(v), reverse=True)
        usado = set([])
        for v in list(V):
            clique_vector = [1 if i in clique_atual else 0 for i in range(G.number_of_nodes())]
            clique_atual.append(v)
            V2 = {u for u in V if u in G[v] and u not in usado}
            if len(V2)+len(clique_atual) > len(best_clique):
                estado = escrever_estado(adj_matriz, clique_vector, v)
                estados_escritos.append(estado)
                matriz_adj_atualizada = atualizar_matriz_adjacencia(adj_matriz, v)
                bb_clique(G, V2, clique_atual, best_clique, arquivo, estados_escritos, matriz_adj_atualizada)
                estados_escritos.pop()
            usado.add(v)
            clique_atual.pop()

In [8]:
import os

def find_max_clique(G, arquivo):
    best_clique = []
    es = []
    V = set(G.nodes())
    V = sorted(V)
    matriz_adj = np.array(nx.adjacency_matrix(G).todense())
    bb_clique(G, V, [], best_clique, arquivo, es, matriz_adj)

    if os.path.exists(f"instancias/{arquivo}"):
      os.remove(f"instancias/{arquivo}")

    return best_clique

In [ ]:
def main():
    graphs = generate_random_graph(2000, 10, 250, 0.4, 0.8)
    i=0
    os.makedirs("instancias", exist_ok=True)
    for G in graphs:
      find_max_clique(G, f"{i}.txt")
      i+=1

if __name__ == "__main__":
    main()

1785
1786
1787
1788
1789
1790
